## Generate t2t data

In [4]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
# from tensor2tensor.utils import registry
# from tensor2tensor import problems

## Generate data

In [5]:
import generate_problem # self-defined 'generate_problem.py' file in the same folder.

USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD'
TMP_DIR = './tmp' # Where data files from internet stored
DATA_LOC = './data' # Where pre-prcessed data is stored
# mkdir -p $DATA_LOC $TMP_DIR

In [6]:
# Init problem T2T object the generated training data
# t2t_problem = problems.problem(PROBLEM)
t2t_problem = generate_problem.OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR) 

INFO:tensorflow:Found vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords
INFO:tensorflow:Skipping generator because outputs files exists at ['./data/online_revew_project_usyd-unshuffled-train-00000-of-00009', './data/online_revew_project_usyd-unshuffled-train-00001-of-00009', './data/online_revew_project_usyd-unshuffled-train-00002-of-00009', './data/online_revew_project_usyd-unshuffled-train-00003-of-00009', './data/online_revew_project_usyd-unshuffled-train-00004-of-00009', './data/online_revew_project_usyd-unshuffled-train-00005-of-00009', './data/online_revew_project_usyd-unshuffled-train-00006-of-00009', './data/online_revew_project_usyd-unshuffled-train-00007-of-00009', './data/online_revew_project_usyd-unshuffled-train-00008-of-00009', './data/online_revew_project_usyd-unshuffled-dev-00000-of-00001']
INFO:tensorflow:Skipping shuffle because output files exist


In [7]:
from tensor2tensor.utils import registry
from tensor2tensor import models
MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files'

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [8]:
from tensor2tensor.utils.trainer_lib import create_hparams
import json
# Init Hparams object from T2T Problem
hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
# hparams.learning_rate_warmup_steps = 45000
# hparams.learning_rate = .4

# Can see all Hparams with code below
print(json.loads(hparams.to_json()))

{'batch_shuffle_size': 512, 'heads_share_relative_embedding': False, 'layer_preprocess_sequence': 'n', 'optimizer_adafactor_clipping_threshold': 1.0, 'force_full_predict': False, 'norm_epsilon': 1e-06, 'multiproblem_label_weight': 0.5, 'multiproblem_max_target_length': -1, 'max_input_seq_length': 0, 'num_heads': 8, 'optimizer_momentum_momentum': 0.9, 'learning_rate_cosine_cycle_steps': 250000, 'initializer_gain': 1.0, 'kernel_height': 3, 'multiproblem_schedule_threshold': 0.5, 'pretrained_model_dir': '', 'optimizer_multistep_accumulate_steps': None, 'filter_size': 2048, 'attention_key_channels': 0, 'learning_rate_decay_staircase': False, 'moe_k': 2, 'optimizer_adafactor_decay_type': 'pow', 'attention_dropout_broadcast_dims': '', 'moe_overhead_eval': 2.0, 'conv_first_kernel': 3, 'multiproblem_fixed_train_length': -1, 'min_length_bucket': 8, 'weight_noise': 0.0, 'multiply_embedding_mode': 'sqrt_depth', 'sampling_method': 'argmax', 'use_fixed_batch_size': False, 'multiproblem_target_eval_

In [9]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
# running time calculation
import timeit
start = timeit.default_timer()

# Initi Run COnfig for Model Training
RUN_CONFIG = create_run_config(
    model_name=MODEL,# Location of where model file is store
    model_dir=TRAIN_DIR
      # More Params here in this fucntion for controling how noften to tave checkpoints and more. 
)

# Create Tensorflow Experiment Object
tensorflow_exp_fn = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
#         t2t_usr_dir='./',
        data_dir='./data', 
        train_steps=40000, # Total number of train steps for all Epochs
        eval_steps=200 # Number of steps to perform for each evaluation
    )

# Kick off Training
tensorflow_exp_fn.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f905e1a1ac8>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f905e1a1b00>, 't2t_device_info': {'num_async_replicas': 1}, 'use_tpu': False, '_device_fn': None, '_protocol': None, '_keep_checkpoint_max': 20, '_save_checkpoints_secs': None, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_session_config': gpu_op

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_35555_512.bottom
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_35555_512.targets_bottom
INFO:tensorflow:Building model body
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:Transforming body output with symbol_modality_35555_512.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 62324224
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_35555_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_35555_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_35555_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-25T04:13:17Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files/model.ckpt-34000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Don

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_35555_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_35555_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_35555_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-25T04:28:16Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Don

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_35555_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_35555_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_35555_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-25T04:43:13Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Don

In [ ]:
# ~4h